<a href="https://colab.research.google.com/github/Habsie/Velib-et-Grand-Paris-Express/blob/main/Villes_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Préparation de la densité par Villes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import plotly.express as px
import matplotlib as plt

# Import des fichiers nécessaires

In [ ]:
#POPULATION 2019_DONNEES MISE A DISPO EN 2022
densité_pop22  = pd.read_excel('drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/grille_densite_7_niveaux_detaille_2022.xlsx')

In [ ]:
#DONNEES INSEE SUR LA SUPERFICIE POUR CALCULER LA DENSITE POP/KM2
df_superficie = pd.read_excel('drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/base_cc_comparateur.xlsx', skiprows=range(0, 5))
df_superficie = df_superficie[['CODGEO','SUPERF']]

# Traitement

## Liste des villes d'idf

In [ ]:
#LISTE DES DEPARTEMENTS D'IDF 
liste_depart =['75','77','78','91','92','93','94','95']

## Extractoin des données de populations 

In [ ]:
#EXTRACTION DES COLONNES UTILES POUR L'ANALYSE
densité_pop22  = densité_pop22[['Code commune','Libellé commune','Population \nmunicipale \n2019']]

In [ ]:
densité_pop22['Code dep'] = densité_pop22['Code commune'].str[0:2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Extraction des villes d'idf

In [ ]:
villes_idf = densité_pop22.loc[densité_pop22['Code dep'].isin(liste_depart) == True]

## Ajout de la superficie dans les villes

In [ ]:
villes_idf = villes_idf.merge(df_superficie,left_on='Code commune', right_on='CODGEO', how='left')

## Ajout de la densité de population

In [ ]:
villes_idf['Densite_pop'] = villes_idf['Population \nmunicipale \n2019']/villes_idf['SUPERF']

## Ajout de la segmentation de la densité

In [ ]:
#FONCTION POUR AFFICHER LA SEGMENTATION EN FONCTION DE LA DENSITE/POPULATION
def segmentation(Densite_pop):
    if Densite_pop <= 10000:
      return 'Segment 1'
    if 10000 < Densite_pop <= 15000:
      return 'Segment 2'
    if 15000 < Densite_pop <= 20000:
      return 'Segment 3'
    if Densite_pop > 20000:
      return 'Segment 4'
    else :   
      return 'Segment 0'
 

In [ ]:
#CREATION DE LA COLONNE SEGMENTATION DENSITE POPULATION
villes_idf['Segmentation_Densite_pop'] = villes_idf.apply(lambda villes_idf: segmentation(villes_idf['Densite_pop']), axis=1)

In [ ]:
#VERIFICATION LISTE SEGMENT
villes_idf['Segmentation_Densite_pop'].unique()

array(['Segment 4', 'Segment 1', 'Segment 3', 'Segment 2'], dtype=object)

# Nettoyage du dataframe final villes

In [ ]:
villes_idf = villes_idf.drop(['CODGEO'],axis = 1)


In [ ]:
villes_idf['Population \nmunicipale \n2019'] = pd.to_numeric(villes_idf['Population \nmunicipale \n2019'])

In [ ]:
villes_idf['Densite_pop'] = villes_idf['Population \nmunicipale \n2019']/villes_idf['SUPERF']

In [ ]:
villes_idf = villes_idf.rename(columns={'Libellé commune': "Commune", "Population \nmunicipale \n2019": "Population 2019", "SUPERF": "Superficie en km2"})

# Extraction de villes en CSV

In [ ]:
villes_idf.loc[villes_idf['Code dep']=='75']

,Code commune,Commune,Population 2019,Code dep,Superficie en km2,Densite_pop,Segmentation_Densite_pop
0,75056,Paris,2165423,75,105.4,20544.810247,Segment 4


In [ ]:
villes_idf.to_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/Villes_idf_densite.csv',index=False)